In [1]:
import pandas as pd
import os

In [3]:
def generate_app_res_data(path, target_path, conflicting_res=""):
    stop_times = pd.read_csv(path+"stop_times.txt",dtype={'trip_id':str, 'stop_id':str})
    stops = pd.read_csv(path+"stops.txt",dtype={'stop_id':str})
    trips = pd.read_csv(path + "trips.txt",dtype={'trip_id':str, 'route_id':str})
    routes = pd.read_csv(path+"/routes.txt", dtype={"route_id":str})

    dicts_stop_name = stops.set_index("stop_id")["stop_name"].to_dict()
    dicts_lat = stops.set_index("stop_id")["stop_lat"].to_dict()
    dicts_lon = stops.set_index("stop_id")["stop_lon"].to_dict()

    # Unique route ids
    unique_routes = routes["route_id"].tolist()

    route_stops = {}

    for ur in unique_routes:
        #print(ur)
        route_ur = trips.loc[trips["route_id"]==ur]
        trip_ids = route_ur["trip_id"].unique()    
        allstops = stop_times.loc[stop_times['trip_id'].isin(trip_ids)]["stop_id"].unique()
        route_stops[ur] = allstops
    
    
    subroute_files = []

    #print(stops.stop_id.unique())
    for ur in unique_routes: 
        #print(ur)
        #sub_route_info = pd.read_csv(target_path+s)
        stop_info = route_stops[ur]
        sub_route_info = pd.DataFrame({"stop_id":stop_info})
        sub_route_info= sub_route_info.astype({"stop_id": str}, errors='raise')

        sub_route_info["stop_name"] = sub_route_info.apply(lambda row: dicts_stop_name[str(row["stop_id"])], axis=1)
        sub_route_info["stop_lat"] = sub_route_info.apply(lambda row: dicts_lat[str(row["stop_id"])], axis=1)
        sub_route_info["stop_lon"] = sub_route_info.apply(lambda row: dicts_lon[str(row["stop_id"])], axis=1)
        #sub_route_info.to_csv(target_path+s, index = False, header=False)
        sub_route_info.to_csv(target_path+"sub_route_info_"+str(ur)+conflicting_res+".csv", index = False, header=False)


In [7]:
generate_app_res_data("../bus_gtfs_wmata/", "./generated_files/WMATA/App_res/")
generate_app_res_data("../rail_gtfs_wmata/", "./generated_files/WMATA/App_res/")
generate_app_res_data("../us-maryland-prince-georges-county-the-bus-gtfs-477/","./generated_files/The_BUS_PG/App_res/", "TB")
generate_app_res_data("../20210826_gtfs/","./generated_files/UMD_SHUTTLE/App_res/")

generate_app_res_data("../CCC GTFS Feed - Fall 2024/", "./generated_files/Charm_city_circulator/App_res/", "CCC")
generate_app_res_data("../harborconnector-md-us/", "./generated_files/Harbor_connector/App_res/", "HC")
generate_app_res_data("../mdotmta_gtfs_marc/", "./generated_files/BCDOT/App_res/")
generate_app_res_data("../mdotmta_gtfs_commuterbus/", "./generated_files/BCDOT/App_res/")
generate_app_res_data("../mdotmta_gtfs_lightrail/", "./generated_files/BCDOT/App_res/")
generate_app_res_data("../mdotmta_gtfs_localbus/", "./generated_files/BCDOT/App_res/")
generate_app_res_data("../mdotmta_gtfs_metro/", "./generated_files/BCDOT/App_res/")

In [8]:
#################################### TO RUN (FOR ALL DC AREA) ###################################
## WMATA
columns_to_keep = ['route_id', 'route_short_name', 'route_long_name', 'type']

stops1 = pd.read_csv("../bus_gtfs_wmata/routes.txt")
stops1["type"] = "localbus"
stops2 = pd.read_csv("../rail_gtfs_wmata/routes.txt")
stops2["type"] = "metro"

frames = [stops1, stops2]
routes1 = pd.concat(frames)

routes1 = routes1[columns_to_keep]
routes1['route_short_name'] ='WMATA - ' + routes1['route_short_name'].astype(str)

## UM SHUTTLE
stops3 = pd.read_csv("../20210826_gtfs/routes.txt")
stops3["type"] = "localbus"
frames = [stops3]
routes2 = pd.concat(frames)
routes2['route_short_name'] ='UM Shuttle - ' + routes2['route_id'].astype(str)
routes2['route_long_name'] = routes2['route_long_name'].str.replace('\d+', '')

routes2 = routes2[columns_to_keep]

## THE BUS

stops4 = pd.read_csv("../us-maryland-prince-georges-county-the-bus-gtfs-477/routes.txt")
stops4["type"] = "localbus"

frames = [stops4]
routes3 = pd.concat(frames)
routes3 = routes3[routes1.columns]
routes3['route_short_name'] ='THE BUS - ' + routes3['route_short_name'].astype(str)
routes3['route_id'] = routes3['route_id']+"TB"

## MARC
stops5 = pd.read_csv("../mdotmta_gtfs_marc/routes.txt")
stops5["type"] = "marc"

frames = [stops5]
routes4 = pd.concat(frames)
routes4 = routes4[columns_to_keep]
routes4['route_short_name'] ='MARC - ' + routes4['route_short_name'].astype(str)


stops1 = pd.read_csv("../mdotmta_gtfs_commuterbus/routes.txt")
stops1["type"] = "localbus"
stops2 = pd.read_csv("../mdotmta_gtfs_lightrail/routes.txt")
stops2["type"] = "lightrail"
stops3 = pd.read_csv("../mdotmta_gtfs_localbus/routes.txt")
stops3["type"] = "localbus"
stops5 = pd.read_csv("../mdotmta_gtfs_metro/routes.txt")
stops5["type"] = "metro"


frames = [stops1,stops2,stops3,stops5]
routes5 = pd.concat(frames)

routes5 = routes5[columns_to_keep]


stops1 = pd.read_csv("../CCC GTFS Feed - Fall 2024/routes.txt")
stops1["type"] = "localbus"
stops1["route_short_name"] = stops1["route_long_name"]

frames = [stops1]
routes6 = pd.concat(frames)
routes6 = routes6[columns_to_keep]

routes6['route_short_name'] ='Charm Circulator - ' + routes6['route_short_name'].astype(str)
routes6['route_id'] = routes6['route_id'].astype(str)+"CCC"


#routes.to_csv('routes_final_UMD_SHUTTLE.csv', header=False, index = False)
routes = pd.concat([routes1,routes2,routes3, routes4, routes5, routes6], sort=False)

routes.to_csv('./generated_files/routes_final_DC.csv', header=False, index = False)


<>:22: SyntaxWarning: invalid escape sequence '\d'
<>:22: SyntaxWarning: invalid escape sequence '\d'
/var/folders/vy/0jm52p7s4t70ydv8vt7dh9qc0000gn/T/ipykernel_6219/567587635.py:22: SyntaxWarning: invalid escape sequence '\d'
  routes2['route_long_name'] = routes2['route_long_name'].str.replace('\d+', '')


In [9]:
######################## TO RUN (For BALTIMORE AREA) #######################
stops1 = pd.read_csv("../mdotmta_gtfs_commuterbus/routes.txt")
stops1["type"] = "localbus"
stops2 = pd.read_csv("../mdotmta_gtfs_lightrail/routes.txt")
stops2["type"] = "lightrail"
stops3 = pd.read_csv("../mdotmta_gtfs_localbus/routes.txt")
stops3["type"] = "localbus"
stops4 = pd.read_csv("../mdotmta_gtfs_marc/routes.txt")
stops4["type"] = "marc"
stops5 = pd.read_csv("../mdotmta_gtfs_metro/routes.txt")
stops5["type"] = "metro"

frames = [stops1,stops2,stops3,stops4,stops5]
routes0 = pd.concat(frames)

columns_to_keep = ['route_id', 'route_short_name', 'route_long_name', 'type']

routes0 = routes0[columns_to_keep]

stops1 = pd.read_csv("../CCC GTFS Feed - Fall 2024/routes.txt")
stops1["type"] = "localbus"
stops1["route_short_name"] = stops1["route_long_name"]

frames = [stops1]
routes1 = pd.concat(frames)


routes1 = routes1[columns_to_keep]
routes1['route_short_name'] ='Charm Circulator - ' + routes1['route_short_name'].astype(str)
routes1['route_id'] = routes1['route_id'].astype(str)+"CCC"

stops = pd.read_csv("../harborconnector-md-us/routes.txt")

stops["type"] = "localbus"
stops["route_short_name"] = stops["route_long_name"]

frames = [stops]
routes2 = pd.concat(frames)

routes2 = routes2[columns_to_keep]
routes2['route_short_name'] ='Harbor Connector - ' + routes2['route_short_name'].astype(str)
routes2['route_id'] = routes2['route_id'].astype(str)+"HC"

routes = pd.concat([routes0, routes1, routes2], sort=False)

routes.to_csv('./generated_files/routes_final_MDOT.csv', header=False, index = False)
